In [1]:
import warnings
warnings.filterwarnings('ignore')

from io import StringIO 
import sys

import json
import pandas as pd
import os
from vega import VegaLite
from ipywidgets import widgets
import numpy as np

from falx.chart import *
from eval_interface import FalxEvalInterface

DATA_DIR = os.path.join("..", "benchmarks")

np.random.seed(2019)

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        self._stderr = sys.stderr
        sys.stdout = self._stringio = StringIO()
        sys.stderr = self._stringio
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout
        sys.stderr = self._stderr
        
def render_vegalite(vis):
    # Render a visualization using vegalite
    VegaLite(vis.to_vl_obj()).display()
    
def print_log(output):
    print_flag = False
    for l in output:
        if "====> " in l:
            print_flag = True
            print("")
        if print_flag:
            print(l)

def run_synthesis(fname, num_samples):
    with open(os.path.join(DATA_DIR, fname), "r") as f:
        data = json.load(f)

    print("# run synthesize {}".format(fname))

    input_data = data["input_data"]
    vis = VisDesign.load_from_vegalite(data["vl_spec"], data["output_data"])
    trace = vis.eval()    
    result = FalxEvalInterface.synthesize(inputs=[input_data], full_trace=trace, num_samples=num_samples)
    print("====> {}".format(fname))
    for p, vis in result:
        print("# table_prog:")
        print("  {}".format(p))
        print("# vis_spec:")
        vl_obj = vis.to_vl_obj()
        data = vl_obj.pop("data")["values"]
        print("    {}".format(vl_obj))
            
def run_wrapper(fname, num_samples=4):
    with Capturing() as output:
        run_synthesis(fname, num_samples)
    print_log(output)

In [2]:
#simple cases
%time run_wrapper('001.json')
%time run_wrapper('002.json')
%time run_wrapper('003.json')
%time run_wrapper('005.json')
%time run_wrapper('006.json')
%time run_wrapper('007.json')
%time run_wrapper('009.json')
%time run_wrapper('010.json')
%time run_wrapper('011.json')
%time run_wrapper('012.json')
%time run_wrapper('022.json')
%time run_wrapper('024.json')
%time run_wrapper('025.json')
%time run_wrapper('027.json')
%time run_wrapper('031.json')
%time run_wrapper('033.json')
%time run_wrapper('035.json')
%time run_wrapper('040.json')
%time run_wrapper('041.json')
%time run_wrapper('042.json')
%time run_wrapper('045.json')
%time run_wrapper('046.json')
%time run_wrapper('047.json')
%time run_wrapper('053.json')
%time run_wrapper('056.json')
%time run_wrapper('058.json')


====> 001.json
# table_prog:
  [[gather(@param0, ['2'])], [gather(@param0, ['3'])]]
# vis_spec:
    {'layer': [{'mark': {'type': 'line', 'opacity': 0.7}, 'encoding': {'x': {'field': 'Quarter', 'type': 'nominal'}, 'y': {'field': 'VALUE', 'type': 'quantitative'}, 'order': {'field': 'Quarter', 'type': 'quantitative'}}, 'transform': [{'filter': 'datum.layer_id == 0'}]}, {'mark': {'type': 'bar', 'opacity': 0.7}, 'encoding': {'x': {'field': 'Quarter', 'type': 'nominal', 'sort': None}, 'y': {'field': 'VALUE', 'type': 'quantitative'}}, 'transform': [{'filter': 'datum.layer_id == 1'}]}], 'resolve': {}}
CPU times: user 732 ms, sys: 28.1 ms, total: 760 ms
Wall time: 763 ms

====> 002.json
# table_prog:
  [[gatherNeg(@param0, ['-1', '-2', '-3'])], [gather(@param0, ['2', '3'])]]
# vis_spec:
    {'layer': [{'mark': {'type': 'bar', 'opacity': 0.7}, 'encoding': {'x': {'field': 'Month', 'type': 'nominal'}, 'y': {'field': 'VALUE', 'type': 'quantitative'}, 'color': {'field': 'KEY', 'type': 'nominal'}}, 


====> 045.json
# table_prog:
  [gatherNeg(@param0, ['-1'])]
# vis_spec:
    {'mark': 'bar', 'encoding': {'x': {'field': 'X1', 'type': 'nominal'}, 'y': {'field': 'VALUE', 'type': 'quantitative'}, 'color': {'field': 'KEY', 'type': 'nominal'}}}
CPU times: user 980 ms, sys: 28.2 ms, total: 1.01 s
Wall time: 1.01 s

====> 046.json
# table_prog:
  [gather(@param0, ['4', '5'])]
# vis_spec:
    {'mark': 'line', 'encoding': {'x': {'field': 'date', 'type': 'nominal'}, 'y': {'field': 'VALUE', 'type': 'quantitative'}, 'color': {'field': 'KEY', 'type': 'nominal'}, 'order': {'field': 'date', 'type': 'quantitative'}}}
CPU times: user 1.48 s, sys: 37.8 ms, total: 1.52 s
Wall time: 1.53 s

====> 047.json
# table_prog:
  [mutate(@param0, -, 2, 3), gatherNeg(mutate(@param0, -, 2, 3), ['-1', '-2'])]
# vis_spec:
    {'mark': 'bar', 'encoding': {'x': {'field': 'label', 'type': 'nominal'}, 'y': {'field': 'VALUE', 'type': 'quantitative'}, 'color': {'field': 'KEY', 'type': 'nominal'}}}
CPU times: user 45.5 s,

In [ ]:
%time run_wrapper('004.json') # ~1min
%time run_wrapper('013.json') # ~30s
%time run_wrapper('014.json') # ~2min
%time run_wrapper('015.json') # ~2min
%time run_wrapper('016.json') # ~30s
%time run_wrapper('017.json') # ~30s
%time run_wrapper('020.json') # ~1min
%time run_wrapper('029.json') # ~2min
%time run_wrapper('030.json') # ~2min
%time run_wrapper('034.json') # ~1min
%time run_wrapper('036.json') # ~1min
%time run_wrapper('043.json') # ~2min
%time run_wrapper('051.json') # ~1min
%time run_wrapper('055.json') # ~30s

In [4]:
#%time run_wrapper('008.json') # cannot solve
#%time run_wrapper('018.json') # cannot solve, filter with two predicates
#%time run_wrapper('019.json') # cannot solve, cut
#%time run_wrapper('021.json') # cannot solve
#%time run_wrapper('028.json') # cannot solve, cut
#%time run_wrapper('037.json') # cannot solve
#%time run_wrapper('039.json') # cannot solve 3 steps
#%time run_wrapper('044.json') # cannot solve, floating point issue
#%time run_wrapper('048.json') # cannot solve, division
#%time run_wrapper('049.json') # cannot solve, 3 steps
#%time run_wrapper('050.json') # cannot solve
#%time run_wrapper('057.json') # cannot solve
#%time run_wrapper('059.json') # cannot solve

In [ ]:
%time run_wrapper('023.json') # new
%time run_wrapper('026.json') # new
%time run_wrapper('052.json') # new
%time run_wrapper('test_1.json') # new
%time run_wrapper('test_2.json') # new

KeyboardInterrupt: 